In [ ]:
# Run this whole cell in Google Colab
# It assumes you uploaded "cell_level_dataset.csv" to /content/
# (Menu: Upload → Files in Colab or use drive mount)

# 1) Install / import
!pip install pandas matplotlib seaborn numpy pyarrow --quiet

import os, json, math
import pandas as pd, numpy as np
import matplotlib.pyplot as plt, seaborn as sns
from collections import Counter
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10,4)

FPATH = "/content/cell_level_dataset.csv"
assert os.path.exists(FPATH), f"File not found at {FPATH} — upload your CSV to /content/ as 'cell_level_dataset.csv'"

# Helper funcs
def try_parse_datetime(series):
    try:
        s = pd.to_datetime(series, errors='coerce')
        if s.isnull().all():
            return None
        return s
    except Exception:
        return None

def detect_time_column(df):
    # heuristics: look for typical names
    candidates = [c for c in df.columns if any(k in c.lower() for k in ("time","timestamp","date","datetime"))]
    if candidates:
        for c in candidates:
            parsed = try_parse_datetime(df[c])
            if parsed is not None:
                return c, parsed
        return candidates[0], None
    # fallback: numeric increasing column
    # choose the numeric column with strictly increasing majority property
    for c in df.select_dtypes(include=[np.number]).columns:
        arr = df[c].values
        if len(arr) > 5:
            diffs = np.diff(arr)
            if np.all(diffs >= 0):  # non-decreasing
                return c, None
    return None, None

def estimate_sampling_rate_from_dt(dt_series):
    # dt_series is pandas datetime index or series of datetimes
    if dt_series is None:
        return None
    dt = dt_series.dropna().sort_values().diff().dt.total_seconds().values
    dt = dt[np.isfinite(dt) & (dt>0)]
    if len(dt) == 0:
        return None
    median_dt = np.median(dt)
    mean_dt = float(np.mean(dt))
    return {"median_seconds": float(median_dt), "mean_seconds": mean_dt, "estimated_hz": 1/median_dt if median_dt>0 else None}

# 2) Load (first few rows w/ fallback encodings)
try:
    df = pd.read_csv(FPATH)
except Exception as e:
    # try alternate encodings / engine
    df = pd.read_csv(FPATH, engine='python', encoding='utf-8', error_bad_lines=False)

print("Loaded shape:", df.shape)
display(df.head(5))

# 3) Basic column / nulls / dtypes
cols = df.columns.tolist()
dtypes = {c: str(df[c].dtype) for c in cols}
null_counts = df.isnull().sum().to_dict()
null_pct = (df.isnull().mean()*100).round(4).to_dict()
unique_counts = {c: int(df[c].nunique(dropna=False)) for c in cols}

print("\nColumns, dtypes, null% and unique counts (top):")
for c in cols:
    print(f"- {c:30} | dtype={dtypes[c]:10} | null%={null_pct[c]:6}% | unique={unique_counts[c]}")

# 4) Detect likely battery / cell id, cycle, soc/soh/rul, voltage/current/temp, time
lower_cols = [c.lower() for c in cols]
found = {}
for key in ["battery_id","cell_id","cycle","soc","soh","rul","voltage","current","temp","time","timestamp","pack_voltage","pack_current"]:
    found[key] = [c for c in cols if key in c.lower()]
# more fuzzy patterns
found["voltage"] += [c for c in cols if any(k in c.lower() for k in ("volt","v")) and c not in found["voltage"]]
found["current"] += [c for c in cols if any(k in c.lower() for k in ("cur","i")) and c not in found["current"]]
found["temp"] += [c for c in cols if any(k in c.lower() for k in ("temp","therm","t")) and c not in found["temp"]]

print("\nDetected candidate columns (examples):")
for k in ["battery_id","cell_id","cycle","soc","soh","rul","voltage","current","temp","time"]:
    print(f"{k:8} -> {found[k][:5]}")

# 5) Detect time column & sampling rate
time_col, parsed_time = detect_time_column(df)
if time_col:
    print("\nTime column detected:", time_col)
    if parsed_time is not None:
        sr = estimate_sampling_rate_from_dt(parsed_time)
        print("Estimated sampling (from time column):", sr)
    else:
        print("Time column exists but couldn't parse to datetimes reliably.")
else:
    print("\nNo obvious time column detected.")

# 6) Identify numeric sensor-like columns (voltage/current/temp) - heuristics
sensor_candidates = []
for c in cols:
    if np.issubdtype(df[c].dtype, np.number):
        # value ranges for typical battery sensors
        mn, mx = df[c].min(skipna=True), df[c].max(skipna=True)
        sensor_candidates.append((c, float(mn), float(mx)))
sensor_candidates_sorted = sorted(sensor_candidates, key=lambda x: x[1])[:50]
print("\nSample numeric columns and ranges (first 20):")
for c,mn,mx in sensor_candidates_sorted[:20]:
    print(f"- {c}: min={mn:.4f}, max={mx:.4f}")

# 7) If 'cycle' present: compute rows per cycle distribution
cycle_col = None
for c in cols:
    if "cycle" == c.lower() or "cycle" in c.lower():
        cycle_col = c
        break

cycle_stats = None
if cycle_col:
    counts = df.groupby(cycle_col).size()
    cycle_stats = {
        "n_cycles": int(counts.shape[0]),
        "rows_per_cycle_median": int(counts.median()),
        "rows_per_cycle_mean": float(counts.mean()),
        "rows_per_cycle_min": int(counts.min()),
        "rows_per_cycle_max": int(counts.max()),
        "rows_per_cycle_counts": counts.value_counts().to_dict()
    }
    print("\nCycle stats:", cycle_stats)
else:
    print("\nNo cycle column detected.")

# 8) Roughly detect if dataset is high-frequency (per-sample) or cycle-aggregated
is_high_freq = False
if cycle_stats:
    # If median rows per cycle > ~10 → likely high-frequency per-cycle timeseries
    if cycle_stats["rows_per_cycle_median"] >= 10:
        is_high_freq = True
    else:
        is_high_freq = False
else:
    # if no cycle col, infer by total rows
    is_high_freq = True if df.shape[0] > 5000 else False

print("\nHeuristic: is_high_frequency_timeseries =", is_high_freq)

# 9) Check for direct labels
labels_present = {}
for label in ["soc","soh","rul","capacity","remaining_capacity","ocv","true_soc"]:
    matches = [c for c in cols if label in c.lower()]
    labels_present[label] = matches
print("\nLabels found (candidates):", labels_present)

# 10) Missing value summary per important column group
important = {
    "voltage": found["voltage"][:6],
    "current": found["current"][:6],
    "temp": found["temp"][:6],
    "time": [time_col] if time_col else []
}
missing_report = {}
for k,clist in important.items():
    for c in clist:
        missing_report[c] = {"null_count": int(df[c].isnull().sum()), "null_pct": float((df[c].isnull().mean()*100).round(4))}
print("\nMissing report (sample):")
for k,v in list(missing_report.items())[:12]:
    print(k, v)

# 11) Quick plot examples (save to /content)
os.makedirs("/content/dataqa_plots", exist_ok=True)

# Histogram of rows-per-cycle if available
if cycle_col:
    plt.figure(figsize=(8,3))
    counts.plot(kind='hist', bins=50)
    plt.title("Rows per cycle distribution")
    plt.xlabel("rows per cycle")
    plt.tight_layout()
    plt.savefig("/content/dataqa_plots/rows_per_cycle_hist.png")
    plt.close()

# Plot example voltage/current vs time if time parsed
if parsed_time is not None:
    for col in (found['voltage'][:2] + found['current'][:2])[:4]:
        if col in df.columns:
            plt.figure()
            plt.plot(parsed_time, df[col])
            plt.title(f"{col} vs time")
            plt.xlabel("time")
            plt.tight_layout()
            plt.savefig(f"/content/dataqa_plots/{col}_vs_time.png")
            plt.close()

# If cycle-based, plot aggregated SOH trend per cycle if SOH exists
if cycle_col and any(found['soh']):
    soh_col = found['soh'][0]
    try:
        per_cycle_soh = df.groupby(cycle_col)[soh_col].mean()
        plt.figure()
        per_cycle_soh.plot(marker='o')
        plt.title("SOH per cycle")
        plt.xlabel("cycle")
        plt.ylabel(soh_col)
        plt.tight_layout()
        plt.savefig("/content/dataqa_plots/soh_per_cycle.png")
        plt.close()
    except Exception:
        pass

# 12) Produce a concise JSON report
report = {
    "loaded_shape": df.shape,
    "columns": cols,
    "dtypes": dtypes,
    "null_pct": null_pct,
    "unique_counts": unique_counts,
    "detected_time_column": time_col,
    "time_parsed": parsed_time is not None,
    "estimated_sampling": sr if 'sr' in locals() else None,
    "cycle_column": cycle_col,
    "cycle_stats": cycle_stats,
    "is_high_frequency_timeseries": is_high_freq,
    "sensor_candidates_sample": sensor_candidates_sorted[:20],
    "labels_found": labels_present,
    "missing_report_sample": dict(list(missing_report.items())[:20])
}

with open("/content/dataset_report.json","w") as f:
    json.dump(report, f, default=lambda o: str(o), indent=2)

# Save a small CSV summary: per-cycle aggregates (if cycle col exists)
if cycle_col:
    agg = df.groupby(cycle_col).agg(['mean','std','min','max'])
    # flatten head
    agg.columns = ['_'.join(x) for x in agg.columns]
    agg.reset_index(inplace=True)
    agg.to_csv("/content/per_cycle_aggregates.csv", index=False)
    print("Saved /content/per_cycle_aggregates.csv")

print("\nJSON report saved to /content/dataset_report.json")
print("plots saved to /content/dataqa_plots/ (if generated)")
print("If per-cycle aggregates generated, saved to /content/per_cycle_aggregates.csv")
print("DONE — please attach or paste the contents of /content/dataset_report.json here (or upload the file).")


Loaded shape: (300000, 9)


,CellID,Cycle,Capacity,Voltage,Current,Temperature,SoH,ICA,DVA
0,1,1,0.000000,4.218664,1.041951,25.141018,0.991443,0.000000,0.000000
1,1,1,0.020408,4.170802,1.020437,25.141018,0.991443,-0.426398,-2.345228
2,1,1,0.040816,4.111785,1.045442,25.141018,0.991443,-0.345801,-2.891833
3,1,1,0.061224,4.093980,1.007348,25.141018,0.991443,-1.146176,-0.872466
4,1,1,0.081633,4.057915,1.023589,25.141018,0.991443,-0.565874,-1.767179



Columns, dtypes, null% and unique counts (top):
- CellID                         | dtype=int64      | null%=   0.0% | unique=20
- Cycle                          | dtype=int64      | null%=   0.0% | unique=300
- Capacity                       | dtype=float64    | null%=   0.0% | unique=50
- Voltage                        | dtype=float64    | null%=   0.0% | unique=300000
- Current                        | dtype=float64    | null%=   0.0% | unique=300000
- Temperature                    | dtype=float64    | null%=   0.0% | unique=6000
- SoH                            | dtype=float64    | null%=   0.0% | unique=6000
- ICA                            | dtype=float64    | null%=   0.0% | unique=300000
- DVA                            | dtype=float64    | null%=   0.0% | unique=300000

Detected candidate columns (examples):
battery_id -> []
cell_id  -> []
cycle    -> ['Cycle']
soc      -> []
soh      -> ['SoH']
rul      -> []
voltage  -> ['Voltage', 'DVA']
current  -> ['Current', 'CellID', '